# Testing for the loss:

In [1]:
import numpy as np
from sklearn.cross_decomposition import CCA
import pandas as pd
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from linear_cca import linear_cca
from torch.utils.data import BatchSampler, SequentialSampler
from DeepCCAModels import DeepCCA
from main import Solver
from utils import load_data, svm_classify
from objectives import cca_loss
try:
    import cPickle as thepickle
except ImportError:
    import _pickle as thepickle
import pickle
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import sklearn.model_selection as model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import pprint

In [134]:
U = np.random.random_sample(1000).reshape(250,4)
V = np.random.random_sample(1000).reshape(250,4)

In [138]:
# Get the first n components:
n_comps = 4
cca = CCA(n_components=n_comps)
U_c_1, V_c_1 = cca.fit_transform(U, V) # Fits and transforms
print(U_c_1.shape, V_c_1.shape)
result1 = np.corrcoef(U_c_1.T, V_c_1.T).diagonal(offset=n_comps) # find how similar the respective components are

l_cca = linear_cca()
l_cca.fit(U,V,n_comps) # Fits, saves the transformation materices (= the weights the matrices should be multiplied with) to obj
U_c_2, V_c_2 = np.matmul(U, l_cca.w[0]), np.matmul(V, l_cca.w[1]) # get the transformed matrices by multiplying
print(U_c_2.shape, V_c_2.shape)
result2 = np.corrcoef(U_c_2.T, V_c_2.T).diagonal(offset=n_comps) # find how similar the respective components are


(250, 4) (250, 4)
(250, 4) (250, 4)


In [139]:
print(result1.shape)
print(result2.shape)
print(result1)
print(result2)
print(cca.score(U, V))
print(cca.score(U, V_c_1))

(4,)
(4,)
[0.20991527 0.1092148  0.07499298 0.01589737]
[0.20991526 0.10921481 0.07499297 0.01589737]
-0.7936051756545341
-0.3383723323179895


In [140]:
c_loss = cca_loss(4, True, torch.device('cpu'))
correlation = c_loss.loss(torch.from_numpy(U),torch.from_numpy(V))
print(correlation)

tensor(-0.4874)
